In [4]:
#Importación de librerias
import numpy as np
import pandas as pd #Leer excel
pd.options.display.max_columns = None #TODO: Buscar utilidad
import os
from openpyxl import load_workbook #Editar Excel
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from PyPDF2 import PdfReader #Leer PDFs

In [5]:
#Variable del Nombre del archivo
fileName = "SALDO.xlsx"
#Variable del Nombre de la hoja
sheetName = 'BASE'
#Variable de fecha de Declaraciones juradas a ejecutar
fecha = '2023-05-01'


#Apertura de excel con pandas para lectura
df = pd.read_excel(fileName, sheet_name=sheetName)

#Apertura de excel con openpyxl para escritura
book = load_workbook(fileName)

#Obtiene el nombre de todas las hojas del documento
nombres_hojas = book.sheetnames

#Asigna formato a las columnas con pandas
df['Período'] = df['Período'].astype(str)
df['Importe'] = df['Importe'].astype(str)

#Creación de clase ClienteConcepto para guardar las que hay que actualizar
class ClienteConcepto:
    def __init__(self, cliente, concepto, linea):
        self.cliente = cliente
        self.concepto = concepto
        self.linea = linea

#Elimina los simbolos para comparar el nombre del cliente con el nombre de la carpeta
def estandarizarnombre(nombreCliente):
    nombreCliente = nombreCliente.replace("&", "")
    nombreCliente = nombreCliente.replace(".", "")
    nombreCliente = nombreCliente.replace("SRL", "")
    nombreCliente = nombreCliente.replace("SA", "")
    nombreCliente = nombreCliente.replace(" ", "")
    return nombreCliente.upper()


In [6]:
#Validación de existencia de hoja BASE
if 'BASE' in nombres_hojas:
    hoja_base = book['BASE']

    #Validación de existencia de hoja LOGS
    if 'LOGS' in nombres_hojas:
        hoja_logs = book['LOGS']
        #Borrado de datos de Logs
        hoja_logs.delete_cols(1)
    else:
        hoja_logs = book.create_sheet(title="LOGS")

    #Variable con la ruta de las carpetas de los clientes
    path_folder = '..\..\..\wns'
    #Listar los nombre de las carpetas de los clientes
    folders = os.listdir(path_folder)

    #Variable de lista vacia de contribuyentes
    contribuyentes = []

    #Recorre las lineas del excel con pandas. i = numero de iteración; f = linea de cliente en el excel
    for i, f in df.iterrows():
        if f.Período == fecha and (f.Importe == '0' or f.Importe == 'nan'):
            #Creación de instancia de ClienteConcepto
            cc = ClienteConcepto(f.Contribuyente, f.Concepto, i+2)
            #Lo agrega a la lista de contribuyentes
            contribuyentes.append(cc)

    index = 0

    #Comienza a recorrer la lista de contribuyentes a actualizar
    for c in contribuyentes:
        index += 1

        año = fecha.split('-')[0]
        mes = fecha.split('-')[1]
        mes_año = f"{mes}-{año}"

        try:
            pdf_path = ''
            for f in folders:
                if estandarizarnombre(f) == estandarizarnombre(c.cliente):
                    pdf_path = str(f'..\..\..\wns/{f}/Impuestos/IVA/{año}/{mes_año}')
                    break

            pdfs = os.listdir(pdf_path)

            exists_pdf = False
            for p in pdfs:
                #Se obtiene la celda de la Hoja Logs lista para escribir
                celda_logs = hoja_logs[f'A{index}']
                #Se obtiene la celda de la Hoja Base lista para escribir
                celda = hoja_base['G' + str(c.linea)]

                if 'DDJJ IVA' in p.upper():
                    exists_pdf = True
                    reader = PdfReader(f'{pdf_path}/{p}')
                    page = reader.pages[0]
                    text = page.extract_text()

                    debito_fiscal = text.split('\nTotal del Crédito Fiscal')[0].split('\nTotal del Débito Fiscal $')[1].replace(' ', '').replace(',', '.')
                    credito_fiscal = text.split('\nAjuste Anual del crédito fiscal por operaciones exentas')[0].split('Total del Crédito Fiscal $')[1].replace(' ', '').replace(',', '.')
                    sldo_a_favor = text.split('\nSaldo del Impuesto a Favor de AFIP')[0].split('Saldo de Libre Disponibilidad a favor del contribuyente del período $')[1].replace(' ', '').replace(',', '.')

                    #Edición de celda de Importe 1-IVA DF
                    if(c.concepto == '1-IVA DF'):
                        celda.value = float(debito_fiscal)
                        celda_logs.value = f'1-IVA DF de Cliente {c.cliente} calculado.'
                    
                    #Edición de celda de Importe 2-IVA CF
                    if(c.concepto == '2-IVA CF'):
                        celda.value = float(credito_fiscal)
                        celda_logs.value = f'2-IVA CF de Cliente {c.cliente} calculado.'

                    #Edición de celda de Importe 3-IMPTO DDO.
                    if(c.concepto == '3-IMPTO DDO.'):
                        celda.value = f"=G{c.linea-2}-G{c.linea-1}"
                        celda_logs.value = f'3-IMPTO DDO. de Cliente {c.cliente} calculado.'

                    #Edición de celda de Importe 4-SLD ACTUAL
                    if(c.concepto == '4-SLD ACTUAL'):
                        celda.value = float('-' + sldo_a_favor) if sldo_a_favor != "0.00" else float(sldo_a_favor)
                        celda_logs.value = f'4-SLD ACTUAL del Cliente {c.cliente} calculado.'           
                                                
            if exists_pdf == False:
                celda_logs.value = f'No se encontró Ningún PDF con el cliente {c.cliente}'           
        

        except FileNotFoundError:
            celda_logs = hoja_logs[f'A{index}']
            celda_logs.value = f'El cliente {c.cliente} no se pudo encontrar dentro de la carpeta WNS'
        except IndexError:
            celda_logs = hoja_logs[f'A{index}']
            celda_logs.value = f'El pdf del cliente {c.cliente} es incorrecto'
        except OSError:
            celda_logs = hoja_logs[f'A{index}']
            celda_logs.value = f'Error, la ruta es incorrecta para {c.cliente}'

    book.save(fileName)
    book.close()

else:
    print("La hoja 'BASE' no existe en el archivo.")
   